# Cycle Times
The "time" a event occurs in a battery time series occurs can have many meanings.
A "date time" is the moment it occured accordingly to a calendar, a "test time" is the elapsed time between, and so on.
The battery data toolkit provides the `TimeEnhancer` to compute the many ways of describing time for a time series,
and the `CycleTimesSummarizer` to produce summaries of times for each cycle

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from batdata.postprocess.timing import CycleTimesSummarizer, TimeEnhancer
from batdata.data import BatteryDataset
from datetime import datetime, timedelta
import pandas as pd

## Load Example Dataset
The "single-resistor" datasets provided with battery-data-toolkit includes only a single cycle. 
We'll duplicate it to provide an example for this cycle.

In [2]:
dataset = BatteryDataset.from_batdata_hdf('../../tests/files/example-data/single-resistor-constant-charge_from-charged.hdf')
dataset.raw_data.tail()

C:\Users\lward\Work\ROVI\battery-data-toolkit\batdata\data.py:90: UserWarning: Metadata was created in a different version of batdata. supplied=0.2.0, current=0.3.2.
  warnings.warn(f'Metadata was created in a different version of batdata. supplied={supplied_version}, current={__version__}.')


,test_time,current,voltage,cycle_number
2396,7188.0,1.0,3.096667,0
2397,7191.0,1.0,3.097500,0
2398,7194.0,1.0,3.098333,0
2399,7197.0,1.0,3.099167,0
2400,7200.0,1.0,3.100000,0


Make a copy of the cycle, increment the `cycle_number` and advance the `test_time`.

In [3]:
cycle_two = dataset.raw_data.copy()
cycle_two['cycle_number'] += 1
cycle_two['test_time'] += cycle_two['test_time'].max() + cycle_two['test_time'].iloc[1]
cycle_two.head()

,test_time,current,voltage,cycle_number
0,7203.0,-1.0,2.899167,1
1,7206.0,-1.0,2.898333,1
2,7209.0,-1.0,2.897500,1
3,7212.0,-1.0,2.896667,1
4,7215.0,-1.0,2.895833,1


As an example, we'll replace the test time with a date time so that we can show to recove the test time using `TimeEnhancer`

In [4]:
timeseries = pd.concat([dataset.raw_data, cycle_two], ignore_index=True)

In [5]:
now = datetime.now()
timeseries['date_time'] = timeseries['test_time'].apply(lambda x: now + timedelta(seconds=x))
timeseries.drop(columns='test_time', inplace=True)

In [6]:
dataset.raw_data = timeseries
dataset.raw_data.head()

,current,voltage,cycle_number,date_time
0,-1.0,2.899167,0,2024-07-09 14:15:30.812201
1,-1.0,2.898333,0,2024-07-09 14:15:33.812201
2,-1.0,2.897500,0,2024-07-09 14:15:36.812201
3,-1.0,2.896667,0,2024-07-09 14:15:39.812201
4,-1.0,2.895833,0,2024-07-09 14:15:42.812201


## Recover the Test Time
The `TimeEnhancer` class computes a test time and cycle time (time since the start of a cycle) given the date time for measurements. It produces:
- `test_time`: Time since the beginning of the time series
- `cycle_time`: Time since the beginning of a cycle

In [12]:
computer = TimeEnhancer()

In [13]:
computer.compute_features(dataset).tail()

,test_time,cycle_time
4797,14391.0,7188.0
4798,14394.0,7191.0
4799,14397.0,7194.0
4800,14400.0,7197.0
4801,14403.0,7200.0


The dataset now contains these additional ways of representing time.

## Compute the Cycle Time
`CycleTimesSummarizer` class computes two features describing entire cycles: 
- `cycle_start`: The time elapsed between first measurement of the cycle and the first measurement for the battery
- `cycle_duration`: The time elapsed between the start of the cycle and the beginning of the next cycle

In [16]:
computer = CycleTimesSummarizer()
computer.column_names

['cycle_start', 'cycle_duration']

Compute them by calling the `compute_features` function.

In [17]:
computer.compute_features(dataset)

,cycle_number,cycle_start,cycle_duration
0,0,0.0,7203.0
1,1,7203.0,7200.0


The first cycle starts at 0 seconds and the second starts 7203 seconds later, yielding a cycle duration of 7203 seconds.

There is no third cycle in our data file, so we only report the duration of the next cycle as the time between its first and last measurement: 7200 seconds.